# ContentPulse - Real-Time Publishing Data Generator

This notebook generates streaming publishing analytics data every 5-10 seconds and writes it to a Databricks volume.


In [1]:
%pip install faker


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run ./contentpulse_config


📰 ContentPulse Project Configuration
   Project: ContentPulse_Publishing_Analytics
   User: kunal.gaurav@databricks.com
   Volume: /Volumes/kunal/publishing/publishing_data
   Table: kunal.publishing.content_engagement_events
🗄️  Lakebase Configuration
   Instance: kunal-gaurav-contentpulse-instance
   Catalog: pg_contentpulse_kunal-gaurav
   Synced Table: kunal.publishing.content_engagement_synced
✅ Catalog 'kunal' ready
✅ Schema 'kunal.publishing' ready
✅ Volume '/Volumes/kunal/publishing/publishing_data' ready
✅ Table 'kunal.publishing.content_engagement_events' ready with Change Data Feed enabled


In [5]:
import random
import time
import json
import datetime
from faker import Faker
import uuid

fake = Faker()

# Article categories for publishing companies (Condé Nast style)
CATEGORIES = [
    "Fashion", "Beauty", "Entertainment", "Technology", 
    "Travel", "Food", "Lifestyle", "News", "Sports", "Business"
]

# Publication names (like Condé Nast publications)
PUBLICATIONS = [
    "Vogue", "GQ", "Vanity Fair", "Wired", "Bon Appétit", 
    "The New Yorker", "Architectural Digest", "Glamour", "Allure", "Self"
]

# Event types
EVENT_TYPES = [
    "page_view",      # 60% of events
    "scroll",         # 20% of events
    "comment",        # 10% of events
    "share",          # 5% of events
    "subscribe"       # 5% of events
]

# Device types
DEVICE_TYPES = ["mobile", "desktop", "tablet"]

# Subscription tiers
SUBSCRIPTION_TIERS = ["free", "basic", "premium", "none"]

# Sample article titles by category
ARTICLE_TITLES = {
    "Fashion": [
        "10 Best Winter Fashion Trends 2024",
        "The Return of Y2K Style: What's Hot Now",
        "Sustainable Fashion: The Future is Green",
        "Paris Fashion Week: Best Runway Moments",
        "Celebrity Style: Red Carpet Looks to Copy"
    ],
    "Beauty": [
        "Skincare Routine for Winter: Expert Tips",
        "Best Makeup Products Under $20",
        "Natural Beauty: Embracing Your True Self",
        "Celebrity Beauty Secrets Revealed",
        "Anti-Aging Treatments That Actually Work"
    ],
    "Entertainment": [
        "Oscar Predictions: Who Will Win Big?",
        "New Streaming Shows You Can't Miss",
        "Behind the Scenes: Hollywood's Biggest Stars",
        "Music Festival Guide 2024",
        "Celebrity Interviews: Exclusive Access"
    ],
    "Technology": [
        "AI Revolution: How ChatGPT is Changing Everything",
        "Best Smartphones of 2024: Complete Guide",
        "Cybersecurity Tips for Remote Workers",
        "The Future of Electric Vehicles",
        "Tech Gadgets You Need Right Now"
    ],
    "Travel": [
        "Hidden Gems: 20 Places to Visit in 2024",
        "Luxury Travel: Best Hotels Worldwide",
        "Budget Travel: See the World for Less",
        "Travel Safety Tips for Solo Travelers",
        "Best Beach Destinations for Summer"
    ],
    "Food": [
        "Easy Recipes for Busy Weeknights",
        "Restaurant Review: NYC's Hottest New Spots",
        "Healthy Eating: Nutritionist's Top Tips",
        "Wine Pairing Guide for Beginners",
        "Street Food Around the World"
    ],
    "Lifestyle": [
        "Work-Life Balance: Finding Your Zen",
        "Home Organization Hacks That Work",
        "Mindfulness and Meditation Guide",
        "Self-Care Sunday: Ideas and Inspiration",
        "Productivity Tips from Successful People"
    ],
    "News": [
        "Breaking: Major Political Development",
        "Economic Outlook: What to Expect in 2024",
        "Climate Change: Latest Research Findings",
        "Social Justice: Movements Making a Difference",
        "Global Health: Updates and Insights"
    ],
    "Sports": [
        "Super Bowl Preview: Who Will Win?",
        "Olympics 2024: Athletes to Watch",
        "Fitness Trends: What's Popular Now",
        "Behind the Game: Athlete Interviews",
        "Sports Technology: The Future of Athletics"
    ],
    "Business": [
        "Startup Success Stories: Lessons Learned",
        "Stock Market Analysis: 2024 Outlook",
        "Leadership Tips from Fortune 500 CEOs",
        "Remote Work: The New Normal",
        "Entrepreneurship Guide for Beginners"
    ]
}

# Cities with coordinates
CITIES = [
    {"city": "New York", "country": "USA", "lat": 40.7128, "lon": -74.0060},
    {"city": "Los Angeles", "country": "USA", "lat": 34.0522, "lon": -118.2437},
    {"city": "London", "country": "UK", "lat": 51.5074, "lon": -0.1278},
    {"city": "Paris", "country": "France", "lat": 48.8566, "lon": 2.3522},
    {"city": "Tokyo", "country": "Japan", "lat": 35.6762, "lon": 139.6503},
    {"city": "Sydney", "country": "Australia", "lat": -33.8688, "lon": 151.2093},
    {"city": "Toronto", "country": "Canada", "lat": 43.6532, "lon": -79.3832},
    {"city": "Berlin", "country": "Germany", "lat": 52.5200, "lon": 13.4050},
    {"city": "Mumbai", "country": "India", "lat": 19.0760, "lon": 72.8777},
    {"city": "São Paulo", "country": "Brazil", "lat": -23.5505, "lon": -46.6333}
]

def generate_reader_event():
    """Generate a single reader engagement event"""
    
    # Weighted event type selection
    event_weights = [60, 20, 10, 5, 5]
    event_type = random.choices(EVENT_TYPES, weights=event_weights)[0]
    
    # Select category and article
    category = random.choice(CATEGORIES)
    article_title = random.choice(ARTICLE_TITLES[category])
    
    # Select location
    location = random.choice(CITIES)
    
    # Device type (mobile is most common)
    device_type = random.choices(
        DEVICE_TYPES, 
        weights=[60, 30, 10]  # Mobile, Desktop, Tablet
    )[0]
    
    # Generate event data
    event = {
        "event_id": f"evt_{uuid.uuid4().hex[:12]}",
        "timestamp": datetime.datetime.utcnow().isoformat(),
        "event_type": event_type,
        "reader_id": f"reader_{random.randint(1000, 99999)}",
        "article_id": f"art_{random.randint(1000, 9999)}",
        "article_title": article_title,
        "category": category,
        "publication": random.choice(PUBLICATIONS),
        "device_type": device_type,
        "country": location["country"],
        "city": location["city"],
        "latitude": location["lat"],
        "longitude": location["lon"],
        "time_on_page_seconds": random.randint(10, 600) if event_type == "page_view" else 0,
        "scroll_depth_percent": random.randint(25, 100) if event_type in ["page_view", "scroll"] else 0,
        "num_comments": 1 if event_type == "comment" else 0,
        "num_shares": 1 if event_type == "share" else 0,
        "ad_impressions": random.randint(2, 8) if event_type == "page_view" else 0,
        "estimated_ad_revenue": round(random.uniform(0.05, 0.50), 2) if event_type == "page_view" else 0.0,
        "is_subscriber": random.choice([True, False]),
        "subscription_tier": random.choice(SUBSCRIPTION_TIERS)
    }
    
    return event

print(f"📰 Starting ContentPulse Data Generation")
print(f"📁 Target Volume: {VOLUME_PATH}")
print(f"⏰ Generating publishing analytics data (every 5 seconds)...")
print(f"📊 Event Types: {', '.join(EVENT_TYPES)}")
print(f"📚 Publications: {', '.join(PUBLICATIONS)}")
print("-" * 80)


📰 Starting ContentPulse Data Generation
📁 Target Volume: /Volumes/kunal/publishing/publishing_data
⏰ Generating publishing analytics data (every 5 seconds)...
📊 Event Types: page_view, scroll, comment, share, subscribe
📚 Publications: Vogue, GQ, Vanity Fair, Wired, Bon Appétit, The New Yorker, Architectural Digest, Glamour, Allure, Self
--------------------------------------------------------------------------------


In [7]:
# Main data generation loop
if __name__ == "__main__":
    count = 0
    
    while True:
        count += 1
        
        # Generate random number of events per batch (simulating burst traffic)
        num_events = random.randint(3, 15)  # 3-15 events every 5 seconds
        
        for _ in range(num_events):
            event = generate_reader_event()
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            file_path = f"{VOLUME_PATH}/CONTENT_EVENTS/event_{timestamp}_{uuid.uuid4().hex[:6]}.json"
            
            # Write to Databricks volume
            dbutils.fs.put(file_path, json.dumps(event), overwrite=True)
        
        # Print progress
        if count % 5 == 0:  # Every 25 seconds (5 batches * 5 seconds)
            print(f"✅ Batch #{count} | Generated {num_events} events | Total: {count * num_events} events")
            print(f"   Latest: {event['publication']} - {event['article_title'][:50]}...")
            print(f"   Event: {event['event_type']} from {event['city']}, {event['country']}")
        
        # Wait 5 seconds before next batch
        time.sleep(5)


✅ Batch #5 | Generated 13 events | Total: 65 events
   Latest: The New Yorker - Cybersecurity Tips for Remote Workers...
   Event: page_view from São Paulo, Brazil
✅ Batch #10 | Generated 13 events | Total: 130 events
   Latest: Vogue - Best Makeup Products Under $20...
   Event: page_view from Mumbai, India
✅ Batch #15 | Generated 9 events | Total: 135 events
   Latest: Bon Appétit - Healthy Eating: Nutritionist's Top Tips...
   Event: page_view from Tokyo, Japan
✅ Batch #20 | Generated 4 events | Total: 80 events
   Latest: Allure - Street Food Around the World...
   Event: scroll from London, UK
✅ Batch #25 | Generated 4 events | Total: 100 events
   Latest: GQ - The Future of Electric Vehicles...
   Event: share from Sydney, Australia
✅ Batch #30 | Generated 3 events | Total: 90 events
   Latest: Self - Music Festival Guide 2024...
   Event: share from Tokyo, Japan
✅ Batch #35 | Generated 13 events | Total: 455 events
   Latest: Glamour - Sports Technology: The Future of Athletics.

KeyboardInterrupt: 